In [1]:
!pip install git+https://github.com/facebookresearch/segment-anything.git


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-1hmx1425
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-1hmx1425
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=6e2fd70b1c8ab6fc94205364a7727f301dd1b046d225d23b4a179a38b2a61ae5
  Stored in directory: /tmp/pip-ephem-wheel-cache-vzkzcatd/wheels/29/82/ff/04e2be9805a1cb48bec0b85b5a6da6b63f647645750a0e42d4
Successfully built segment_anything


In [2]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -O sam_vit_h.pth


--2025-08-27 10:21:54--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.14, 3.163.189.108, 3.163.189.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h.pth’

sam_vit_h.pth       100%[===================>]   2.39G   130MB/s    in 15s     

2025-08-27 10:22:09 (163 MB/s) - ‘sam_vit_h.pth’ saved [2564550879/2564550879]



In [3]:
import os
import json
import numpy as np
import cv2
from segment_anything import sam_model_registry, SamPredictor
import matplotlib.pyplot as plt

# ----------------------
# Load SAM
# ----------------------
sam_checkpoint = "sam_vit_h.pth"   # or vit_b, vit_l depending on speed
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device="cuda")  # move SAM model to GPU
predictor = SamPredictor(sam)

# ----------------------
# Load YOLOv8 JSON detections
# ----------------------
with open("Flux-Dev_yolov8_detections.json", "r") as f:
    detections = json.load(f)

# ----------------------
# Base folder for images
# ----------------------
base_image_path = "/content/drive/MyDrive/Flux-Dev"

final_results = []  # to store SAM-refined results

for item in detections:
    # Prepend base path
    image_path = os.path.join(base_image_path, item["image"])
    image = cv2.imread(image_path)

    if image is None:
        print(f" Image not found: {image_path}")
        continue

    predictor.set_image(image)

    refined_boxes = []
    for det in item["boxes"]:
        box = det["box"]  # [x1, y1, x2, y2]
        score = det["score"]
        label = det["label"]

        # Convert box to NumPy array (required by SAM)
        input_box = np.array(box)

        # Get SAM mask
        masks, scores, _ = predictor.predict(
            box=input_box[None, :],  # shape (1, 4)
            multimask_output=False
        )

        refined_boxes.append({
            "box": box,
            "score": score,
            "label": label,
            "mask": masks[0].astype("uint8").tolist()  # convert numpy -> list for JSON
        })

    final_results.append({
        "image": item["image"],  # keep original filename in JSON
        "objects": refined_boxes
    })



# ----------------------
# Save final JSON with masks
# ----------------------
with open("Flux-Dev_yolov8_sam_refined.json", "w") as f:
    json.dump(final_results, f)
